<a href="https://colab.research.google.com/github/JeisonTantachuco/CursoGEE/blob/main/module06/05_NDVI_NDWI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio Propuesto: Elaborar un Mapa de Índice espectral (NDVI, NDSI, NDWI, etc.)

El resultado debe ser un Mapa que tenga los datos de algun índice diferencia normalizada calculado usando Google Earth Engine, para lo cual se debe tener en cuenta:

1) Calcular el índice desde Google Earth Engine - Colab y descargarlo a drive para poder usarlo de forma externa, la zona que se trabaje debe ser diferente a la trabajada en clase.

2) El mapa debe tener todos los elementos cartográficos: Grillado de coordenadas, orientación al norte, Escala Gráfica y Numérica, Leyenda, Membrete y Título de mapa.

3) El mapa puede ser generado en cualquier software GIS.

**La tarea se considerara como concluida luego de enviar lo solicitado en el ejercicio 1, y el mapa generado en este ejercicio al correo indicado en la plataforma: tareas@mastergis.com**

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project = 'ee-jtantaroman')

In [33]:
#@title mapdisplay: Crea mapas interactivos usando folium
import folium
def mapdisplay(center, dicc, Tiles="OpenStreetMap", zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright, Mapbox Control Room, Stamen Terrain, etc.
    :param zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    center = center[::-1]
    mapViz = folium.Map(location=center, tiles=Tiles, zoom_start=zoom_start)

    # Agregar Google o Esri tiles
    esri_satellite = folium.TileLayer(
        tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
        attr="Esri",
        name="Esri Satellite",
        overlay=True,
        control=True
    )
    esri_hybrid = folium.TileLayer(
        tiles="https://server.arcgisonline.com/ArcGIS/rest/services/Reference/World_Boundaries_and_Places/MapServer/tile/{z}/{y}/{x}",
        attr="Esri",
        name="Esri Hybrid",
        overlay=True,
        control=True
    )

    esri_satellite.add_to(mapViz)
    esri_hybrid.add_to(mapViz)

    for k, v in dicc.items():
        if ee.image.Image in [type(x) for x in v.values()]:
            folium.TileLayer(
                tiles=v["tile_fetcher"].url_format,
                attr='Google Earth Engine',
                overlay=True,
                name=k
            ).add_to(mapViz)
        else:
            folium.GeoJson(
                data=v,
                name=k
            ).add_to(mapViz)

    mapViz.add_child(folium.LayerControl())
    return mapViz

#1. Cargar datos vectoriales

In [ ]:
# Llamando al feature collection que contiene los limites de los paises
name_country = 'Bolivia'
countries = ee.FeatureCollection("USDOS/LSIB/2017")
roi = countries.filter(ee.Filter.eq('COUNTRY_NA',name_country))
# Calculando el centroide para la visualizacion
center = roi.geometry().centroid().coordinates().getInfo()

In [ ]:
mapdisplay(center, {'Country':roi.getMapId()}, Tiles="OpensTreetMap",zoom_start=6)

#2. Cargar datos raster (Imagenes)

In [ ]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filterDate('2020-01-01', '2021-12-31')\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))\
    .map(mask_s2_clouds)\
    .filterBounds(roi)

In [ ]:
visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

mapdisplay(center, {'sentinel2':sentinel2.getMapId(visualization)}, Tiles="OpensTreetMap",zoom_start=6)

#3. Calculo del índice normalizado
Utiliza .normalizedDifferece para realizar este ejercicio

In [ ]:
#Realizamos el mosaico para convertir el image collection a image
sentinel2Mosaic = sentinel2.mosaic()
#Calculo del NDWI
ndwi = sentinel2Mosaic.normalizedDifference(['B3', 'B8']).rename('NDWI')
#Clip al NDWI
ndwi_clip = ndwi.clip(roi)

In [42]:
#Realizamos una mediana
sentinel2Median = sentinel2.median()
#Calculo del NDWI
ndwiMedian = sentinel2Median.normalizedDifference(['B3', 'B8']).rename('NDWI')
#Clip al NDWI
ndwi_clipMedian = ndwiMedian.clip(roi)

In [43]:
visualization = {
    'min': -0.5,
    'max': 0.5,
    'palette': ['white', 'lightblue', 'blue', 'darkblue']
}

mapdisplay(center, {'NDWI':ndwi_clip.getMapId(visualization),
                    'NDWI_median':ndwi_clipMedian.getMapId(visualization)}, Tiles="OpensTreetMap",zoom_start=6)

#4. Descargar los resultados (De Google Earth Engine a Google Drive)
**ee.batch.Export.table.toDrive()**: Guarda FeatureCollection como shapefile en Google Drive.

**ee.barch.Export.image.toDrive()**: Guarda Imagnees como GeoTIFF en Google Drive.